We are going to create a Folium map that shows 10 major U.S. cities as markers in the map. When clicking each marker, a pop-up window appears and shows an interactive Plotly chart. The chart shows the housing market trends for the past 4 years for the city, with a range slider allowing users to select different time periods. The Awesome Folium map we are going to create looks like this:

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import folium
import branca

#read in housing market data--takes a bit, be patient...
df = pd.read_csv('data/redfin-extract.csv')

### Data Wrangling
There are a few things to notice in the data above. The field ‘period_begin’ is a string and may cause some problems later on when using it to plot a time-series chart. Therefore, let’s change it to the ‘date’ type and rename it to ‘snapshot_month’.

Both ‘median_sale_price’ and ‘homes_sold’ fields have one decimal place which is unnecessary so let’s change both of them to integer type. We will also create a new field called ‘median_sale_price_formatted’ which shows the numbers with the thousands separator.

In [ ]:
#Data Wrangling and Cleaning
df['snapshot_month'] = pd.to_datetime(df['period_begin'])
df["median_sale_price"]=df["median_sale_price"].astype('int')
df["homes_sold"]=df["homes_sold"].astype('int')
df.loc[:, "median_sale_price_formatted"] = df["median_sale_price"].map('{:,d}'.format) 
df.rename({'region': 'city'}, axis=1, inplace=True)
df=df.sort_values("snapshot_month")

In [ ]:
df

Lastly, let’s create a new field called ‘text’. Why do we need it? By default, Plotly shows a tooltip that displays some basic information when users move their mouses onto each data point. However, the default option is very limiting and not very visually appealing so we want to create a customized tooltip in order to control what and how the information is to be shown in the hover-over tooltip.

In [ ]:
hover_text = []
for index, row in df.iterrows():
    hover_text.append(('<b>{city}</b><br><br>'+
                      'Snapshot Month: {period_begin}<br>'+
                      'Homes Sold: {homes_sold}<br>'+ 
                      'Median Sales Price ($): {median_sale_price_formatted}<br>'
                       ).format(
                       city=row['city'],
                       period_begin=row['period_begin'],
                       homes_sold=row['homes_sold'],
                       median_sale_price_formatted=row['median_sale_price_formatted']
                      ))
df['text'] = hover_text

### Create a Dual Y-Axis Plotly Chart
Our data is ready and let’s start plotting. Remember our ultimate goal is to create an interactive Plotly chart for each city and then pass each chart to its corresponding marker in the Folium map. We will need to use for-loop and HTML iframe to achieve that.

Before we get to that, let’s warm up a little bit by creating a double y-axis Plotly chart for one city only. We can choose Atlanta and plot the ‘homes_sold’ metric in a bar graph and the ‘median_sale_price’ metric in a line graph and combine the two graphs into one combo chart with dual y-axes. We’ll also add a customized hover-over tooltip to the combo chart as well as a range slider/selector.

Using Plotly Graph_Objects, we first create a figure using make_subplots() so that the figure can have two y-axes with different scales. Then we add two traces to the figure, one representing a bar graph and the other a line graph. Lastly, we update the figure using update_layout() method to customize the chart title, legend, x and y axes, etc.

If you are new to Plotly, you can read this article below to learn the basics of Plotly and Plotly Graph_Objects. By doing so, you will have a much easier time following and understanding the code that’s coming next!

For ease of demonstration, I’m going to present the whole code, then I’m going to comment part by part of the code so that you can have a good understanding of how this works.

In [ ]:
df_city=df[(df['city'] =='Atlanta, GA')]
fig=make_subplots(specs=[[{"secondary_y":True}]])

fig.add_trace(                           #Add a bar chart to the figure
        go.Bar(
        x=df_city['snapshot_month'],
        y=df_city['homes_sold'],
        name="Homes Sold",
        hoverinfo='none'                 #Hide the hoverinfo
        ),
        secondary_y=False)               #The bar chart uses the primary y-axis (left)

fig.add_trace(                           #Add the second chart (line chart) to the figure
    go.Scatter(
    x=df_city['snapshot_month'],
    y=df_city['median_sale_price'],
    name="Median Sale Price ($)",
    mode='lines',
    text=df_city['text'],               
    hoverinfo='text',                   #Pass the 'text' column to the hoverinfo parameter to customize the tooltip
    line = dict(color='firebrick', width=3)#Specify the color of the line
     ),
    secondary_y=True)                   #The line chart uses the secondary y-axis (right)

fig.update_layout(hoverlabel_bgcolor='#DAEEED',  #Change the background color of the tooltip to light gray
             title_text="Housing Market Trends: Atlanta, GA", #Add a chart title
             title_font_family="Times New Roman",
             title_font_size = 20,
             title_font_color="darkblue", #Specify font color of the title
             title_x=0.5, #Specify the title position
             xaxis=dict(
                    tickfont_size=10,
                    tickangle = 270,
                    showgrid = True,
                    zeroline = True,
                    showline = True,
                    showticklabels = True,
                    dtick="M1", #Change the x-axis ticks to be monthly
                    tickformat="%b\n%Y"
                    ),
             legend = dict(orientation = 'h', xanchor = "center", x = 0.72, y= 1), #Adjust legend position
             yaxis_title='# Homes Sold',
             yaxis2_title='Median Sales Price ($)')

fig.update_xaxes(
rangeslider_visible=True,
rangeselector=dict(
    buttons=list([
        dict(count=1, label="1m", step="month", stepmode="backward"),
        dict(count=6, label="6m", step="month", stepmode="backward"),
        dict(count=1, label="YTD", step="year", stepmode="todate"),
        dict(count=1, label="1y", step="year", stepmode="backward"),
        dict(step="all")
    ]))
)

fig.show()

### Use for-loop to Create a Plotly Chart for Each City and Export it to HTML
Now coming back to our end goal, we’ll need to use a for-loop to automate the process above and create a Plotly chart for each city and export them to HTML files — one HTML file for each city! To help you understand how this works, I am going to use the same strategy: I’ll first present the whole code and then explain it part by part.

In [ ]:
figs = {} # Create an empty figure to which we will add all the plotly figures
city_list=[] #Create an empty list to which we will append all the cities
html_list=[] #Create an empty list to which we will append all the exported html files
cities = df['city'].unique() # Get the list of city names
for i, city in enumerate(cities, start = 0):
    city_list.append(city)
    html_list.append('fig'+str(i)+'.html')
    df_city = df[df['city']==city]
    
    ####Insert the code (line 2-55) from previous section here. Change line 26 to: title_text="Housing Market Trends: "+cities[i]. Also, when copy-and-paste make sure you place the code within the for-loop (properly indented)#########
    
    figs['fig'+str(i)] = fig
    fig.write_html('data/fig'+str(i)+".html")

df1=pd.DataFrame(city_list,columns =['city'])
df2=pd.DataFrame(html_list,columns =['html_file'])
df3=pd.concat([df1, df2], axis=1)
lat_lon=pd.read_csv('data/cities_lat_lon.csv') #Add the latitude and longitude for each city so that we can later on add them to markers
df_final=df3.merge(lat_lon, on='city', how='left')
df_final #Create a final dataframe that has city names, html file names,  latitude and longitude.

In [ ]:
lat_lon

In [ ]:
location = df_final['Latitude'].mean(), df_final['Longitude'].mean()
m = folium.Map(location=location,zoom_start=4)

for i in range(0,len(df_final)):
    html="""
    <iframe src=\"""" + df_final['html_file'][i] + """\" width="850" height="400"  frameborder="0">    
    """
    
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df_final['Latitude'].iloc[i],df_final['Longitude'].iloc[i]],
                  popup=popup,icon=folium.Icon( icon='home', prefix='fa')).add_to(m)
    
m.save("index.html")